In [2]:
# 名前マンぐリング
# 属性名の頭に __ をつける
# 継承時の名前衝突を避ける仕組み
# パブリックでないだけの属性の表現には _ を用いる

class MyClass:
    __secret_value = 1

instance_of = MyClass()
instance_of.__secret_value

AttributeError: 'MyClass' object has no attribute '__secret_value'

# 3.3.1 ディスクリプタ

オブジェクトの属性が参照されたときの挙動をカスタマイズできる
ディスクリプタプロトコルを構成する4つの特殊メソッド
-  __set__() 属性がセットされるときに呼ばれる セッター
- __get__()  属性が読み込まれるときに呼ばれる ゲッター
- __delete__() 属性に対してDelが実行されたときに呼ばれる
- __set_name__() py36から．ディスクプリたが他のクラスに追加されたときに，対象のクラスと属性名を伴って呼ばれる．

get,setの含まれるディスクリプタはデータディスクリプタ，getのみの場合は非データディスクプリプタ
オブジェクトの持つ特殊なgetattribute()メソッドが属性を参照するときにこれらのプロトコルを呼び出す．


In [6]:
class RevealAccess(object):
    """通常と同じようにデータの設定とアクセスを行うが，アクセスされたログメッセージを残すデータディスクリプタ"""
    def __init__(self,initval=None,name = "var"):
        self.val= initval
        self.name = name
    def __get__(self,obj,objtype):
        print("取得：",self.name)
        return self.val
    def __set__(self,obj,val):
        print("更新：",self.name)
        self.val = val
class MyClass(object):
    x = RevealAccess(10,"変数 x")
    y = 5

# インスタンス属性が取得されると常に__get__が呼び出される
# 値を設定する時にはSetが呼び出される
m = MyClass()
m.x

取得： 変数 x


10

In [8]:
m.x = 20
m.x

更新： 変数 x
取得： 変数 x


20

In [9]:
m.y

5

In [11]:
def function():pass
print(hasattr(function,"__get__"))
print(hasattr(function,"__set__"))

True
False


In [14]:
# 現実世界のサンプル - 属性の遅延評価
class InitOnAccess:
    def __init__(self,klass,*args,**kwargs):
        self.klass= klass
        self.args = args
        self.kwargs = kwargs
        self._initialized = None
        
    def __get__(self,instance,owner):
        if self._initialized is None:
            print("初期化")
            self._initialized =self.klass(*self.args,**self.kwargs)
        else :
            print("キャッシュ済み")
        return self._initialized

In [15]:
class MyClass:
    laizy_initialized = InitOnAccess(list,"argment")

m = MyClass()
m.laizy_initialized
    

初期化


['a', 'r', 'g', 'm', 'e', 'n', 't']

In [16]:
m.laizy_initialized


キャッシュ済み


['a', 'r', 'g', 'm', 'e', 'n', 't']

In [17]:
# デコレータかつデータディスクリプタ
# 一度しか実行されず，実行後は関数の返り値で暮らす属性が書き換えられる．
# オブジェクトの初期化がグローバルなアプリケーションの状態や文脈に依存するために
# import 時に初期化できない要求を満たすことが出来る．
class lazy_property(object):
    def __init__(self,function):
        self.fget = function
    def __get__(self,obj,cls):
        value = self.fget(obj)
        setattr(obj,self.fget.__name__,value)
        return value
    
    

In [19]:
class lazy_class_attribute(object):
    def __init__(self, function):
        self.fget = function

    def __get__(self, obj, cls):
        value = self.fget(obj or cls)
        # note: storing in class object not its instance
        #       no matter if its a class-level or
        #       instance-level access
        setattr(cls, self.fget.__name__, value)
        return value


class MyComplexClass:

    @lazy_class_attribute
    def evaluated_only_once(self):
        print("Evaluation of a method!")
        return sum(x ** 2 for x in range(200))


instance = MyComplexClass()

print("First access to attribute at instance level")
print("instance.evaluated_only_once =",
      instance.evaluated_only_once,
      '\n')

print("Next access to attribute at instance level")
print("instance.evaluated_only_once =",
      instance.evaluated_only_once,
      '\n')

print("Access to attribute at class level")
print("MyComplexClass.evaluated_only_once =",
      MyComplexClass.evaluated_only_once,
      '\n')

print("Access to attribute from completely new instance")
print("MyComplexClass().evaluated_only_once =",
      MyComplexClass().evaluated_only_once,
      '\n')

First access to attribute at instance level
Evaluation of a method!
instance.evaluated_only_once = 2646700 

Next access to attribute at instance level
instance.evaluated_only_once = 2646700 

Access to attribute at class level
MyComplexClass.evaluated_only_once = 2646700 

Access to attribute from completely new instance
MyComplexClass().evaluated_only_once = 2646700 



## 3.3.2 プロパティ
 属性 と それを処理するメソッドをリンクさせる組み込みディスクリプタ型を提供する
 
 fget引数とfset,fdel,docのオプション引数をとる
 
 簡単にディスクリプタをかけるが，継承時には注意が必要


In [34]:
# メンテナンスの面でPropertyはデコレータとして利用するのが吉
# function.setter 

class Rectangle:
    def __init__(self,x1,y1,x2,y2):
        self.x1,self.y1 = x1,y1
        self.x2,self.y2 = x2,y2
    
    # widthをプロパティでデコレーション
    # widthのgetプロパティ
    @property
    def width(self):
        """左辺から測定した短形の幅"""
        return self.x2 - self.x1
    
    # widthのsetプロパティでデコレーション
    @width.setter
    def width(self,value):
        self.x2 = self.x1 + value
    
    @property
    def height(self):
        """上辺から測定した短形の幅"""
        return self.y2 - self.y1
    
    @height.setter
    def height(self,value):
        self.y2 = self.y1 + value
    
    def __repr__(self):
        return "{}({},{},{},{})".format(self.__class__.__name__,self.x1,self.y1,self.x2,self.y2)

rectangle = Rectangle(10,10,25,34)
rectangle.width,rectangle.height

(15, 24)

In [35]:
rectangle.width = 100
rectangle

Rectangle(10,10,110,34)

In [36]:
help(rectangle)

Help on Rectangle in module __main__ object:

class Rectangle(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, x1, y1, x2, y2)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  height
 |      上辺から測定した短形の幅
 |  
 |  width
 |      左辺から測定した短形の幅



# 3.3.3 スロット
__slots__という名前で属性名のリストをセットすることでクラスをインスタンス化するときに__dict__が生成されなくなる

属性が少ないクラスにおいて，すべてのインスタンスで__dict__を作らないことで，メモリ消費を節約できる

In [ ]:
class Frozen:
    __slots__=["Ice","Cream"]
"__dict__"in dir(Frozen)

In [ ]:
"Ice"in dir(Frozen)

In [ ]:
frozen = Frozen()
frozen.Ice = True

In [ ]:
# 動的に追加できないよ
frozen.Icy=True

In [51]:
class Frozen:
    __slots__=["Ice","Cream"]
"__dict__"in dir(Frozen)

False

In [52]:
"Ice"in dir(Frozen)

True

In [53]:
frozen = Frozen()
frozen.Ice = True

In [55]:
# 動的に追加できないよ
frozen.Icy=True

AttributeError: 'Frozen' object has no attribute 'Icy'